In [1]:
from adapters import AutoAdapterModel, AdapterConfig
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
model = AutoAdapterModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
config = AdapterConfig.load("houlsby", non_linearity="gelu", reduction_factor=2)


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
ruta_del_archivo = 'E:/Git Hub Inteligencia Artificial/Practica Modelos LLM/notebooks/data.xlsx'
dataframe = pd.read_excel(ruta_del_archivo)

# Muestra las primeras filas del DataFrame para verificar
print(dataframe.columns)

Index(['Unnamed: 0', 'Preguntas', 'Respuestas', 'Clasificación'], dtype='object')


In [3]:
dfChatColumns = ['Preguntas','Respuestas']
dfChat = dataframe[dfChatColumns]

In [4]:
dfChat

,Preguntas,Respuestas
0,Hola,"Hola, en que puedo ayudarte?"
1,¿Cuantos programas tienes?,"Indicame que area quisieras:Tecnológicas, Cien..."
2,Que cursos vendes,"Indicame que area quisieras:Tecnológicas, Cien..."
3,Que cursos tienes,"Indicame que area quisieras:Tecnológicas, Cien..."
4,Cursos disponibles,"Que área deseas:Tecnológicas, Ciencias Económi..."
...,...,...
58,Quisiera saber acerca de tus cursos de formación,"Que área deseas:Tecnológicas, Ciencias Económi..."
59,Cuál es la gama de cursos que tienes disponible?,"Indicame que area quisieras:Tecnológicas, Cien..."
60,¿Cuál es la gama de cursos que tienes disponible?,"Que área deseas:Tecnológicas, Ciencias Económi..."
61,Estoy buscando detalles sobre tus programas ed...,"Indicame que area quisieras:Tecnológicas, Cien..."


In [8]:
class DatasetChatbt(Dataset):
    def __init__(self,data,tokenizer, max_length=128):
        self.dataset=data
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return (len(self.dataset))-1
    def __getitem__(self, index):
        input = self.dataset['Preguntas'][index]
        target = self.dataset['Respuestas'][index]
        inpTokn = self.tokenizer.encode(input, max_length=self.max_length,padding="max_length",add_special_tokens=True,truncation=True)
        labelTokn = self.tokenizer.encode(target, max_length=self.max_length,padding="max_length",add_special_tokens=True,truncation=True)
        inpToknR = torch.tensor(inpTokn, dtype=torch.long)
        labelTokn = torch.tensor(labelTokn, dtype=torch.long)
        mask = np.ones(128)
        return {'input_ids':inpToknR,'labels': labelTokn, 'attention_mask':mask}

In [6]:
len(dfChat)

63

In [9]:
datasetChatbot = DatasetChatbt(dfChat, tokenizer)

In [48]:
for i in datasetChatbot:
    print(i)

{'input_ids': tensor([  101, 20220, 10330,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

KeyError: 63

In [10]:
#model.load_adapter("es/wiki@ukp", config=config)

In [11]:
#model.adapter_summary()

In [12]:
#model.add_adapter("es/wiki@up")

In [13]:
#model.adapter_summary()

In [14]:
#model.set_active_adapters('es/wiki@up')

In [15]:
#model.adapter_summary()

In [16]:
model.add_causal_lm_head("chatbot", activation_function='gelu')

In [17]:
model.add_classification_head("recommend_client",num_labels=2)
#The weights of this classification head can be stored together with the adapter weights to allow for a full reproducibility. The method call model.set_active_adapters("sst-2") registers the sst-2 adapter as a default for training. This also supports adapter stacking and adapter fusion!
#model.set_active_adapters("es/wiki@ukp")

In [18]:
model.get_labels_dict('recommend_client')

{0: 'LABEL_0', 1: 'LABEL_1'}

In [20]:
#model.adapter_summary()

In [21]:
#model

In [22]:
from adapters import init
#init(model)
model.add_adapter('chatbot')
model.set_active_adapters("chatbot")
model.train_adapter("chatbot")

In [25]:
from transformers import TrainingArguments
from adapters import AdapterTrainer
training_args = TrainingArguments(
  output_dir="./examples",
  do_train=True,
  remove_unused_columns=False,
  learning_rate=5e-4,
  num_train_epochs=10,
)
trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=datasetChatbot,
    )



In [71]:
trainer.train() #con 20 reps

Step,Training Loss


TrainOutput(global_step=160, training_loss=0.6513864040374756, metrics={'train_runtime': 1925.7349, 'train_samples_per_second': 0.644, 'train_steps_per_second': 0.083, 'total_flos': 83092581734400.0, 'train_loss': 0.6513864040374756, 'epoch': 20.0})

In [24]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=120, training_loss=2.859299977620443, metrics={'train_runtime': 1586.5276, 'train_samples_per_second': 0.586, 'train_steps_per_second': 0.076, 'total_flos': 63250665984000.0, 'train_loss': 2.859299977620443, 'epoch': 15.0})

In [26]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=80, training_loss=1.9410478591918945, metrics={'train_runtime': 680.8789, 'train_samples_per_second': 0.911, 'train_steps_per_second': 0.117, 'total_flos': 42167110656000.0, 'train_loss': 1.9410478591918945, 'epoch': 10.0})

In [27]:
model.save_adapter("adapter_chatbot", "chatbot")

In [82]:
tokn= tokenizer.encode("Que cursos tienes", max_length=128,padding="max_length",add_special_tokens=True,truncation=True)

In [63]:
for i in datasetChatbot:
    print(i)

{'input_ids': tensor([  101, 20220, 10330,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

KeyError: 63

In [ ]:
PREFIX= "Que cursos tienes"
encoding = tokenizer(PREFIX, return_tensors="pt")
output_sequence = model.generate(
  input_ids=tokenizer.encode("Que cursos tienes", max_length=128,padding="max_length",add_special_tokens=True,truncation=True),
  do_sample=True,
  num_return_sequences=5,
  max_length = 50,
)



In [ ]:
datasetClassification = DatasetClassif()

In [5]:
model.add_adapter('recommendation')
model.set_active_adapters("recommendation")
model.train_adapter("recommendation")

In [ ]:
from transformers import TrainingArguments
from adapters import AdapterTrainer, Seq2SeqAdapterTrainer, AdapterConfig
training_args = TrainingArguments(
  output_dir="./examples",
  do_train=True,
  remove_unused_columns=False,
  learning_rate=5e-4,
  num_train_epochs=3,
  
)
trainer = AdapterTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=datasetClassification,
    )

In [8]:
for i in model.adapters_config:
    print(i)

recommendation
chatbot


In [18]:
from datasets import Dataset
from transformers import AutoTokenizer

# Supongamos que tienes una lista de textos y sus correspondientes etiquetas
texts = ["Texto 1", "Texto 2", "Texto 3"]
labels = [0, 1, 0]  # 0 o 1 representan las clases

# Crea un diccionario con los datos
data = {"text": texts, "label": labels}

# Crea un objeto Dataset
dataset = Dataset.from_dict(data)

# Descarga el tokenizador preentrenado
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokeniza los textos y añade las etiquetas
#def tokenize_function(examples):
#    return tokenizer(examples["text"], padding=True, truncation=True)
num1 = 0
def prueba(ex):
    num = 3
    return ex + "-"+num
tokenized_datasets = dataset.map(prueba, batched=True)

# Imprime las primeras filas del conjunto de datos tokenizado
print(tokenized_datasets["train"][0])

# Salida esperada:
# {'attention_mask': [1, 1, 1, ...], 'input_ids': [101, ..., 102], 'label': 0}


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

TypeError: unsupported operand type(s) for +: 'LazyBatch' and 'str'

In [19]:
tokenized_datasets

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3
})

In [16]:
for i in tokenized_datasets:
    print(i)

{'text': 'Texto 1', 'label': 0, 'input_ids': [101, 3793, 2080, 1015, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
{'text': 'Texto 2', 'label': 1, 'input_ids': [101, 3793, 2080, 1016, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
{'text': 'Texto 3', 'label': 0, 'input_ids': [101, 3793, 2080, 1017, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
